<a href="https://colab.research.google.com/github/reza-mohammadi-2040/Indoor-Localization-using-Stacked-Denoising-Autoencoder/blob/main/Indoor_Localization_vSDAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 0: Load library

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout ,Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Flatten, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error


# Step 1: Load and preprocess UJIIndoorLoc dataset

In [5]:
# Load dataset (should be manually uploaded to Colab)
data_training = pd.read_csv("/content/drive/MyDrive/AI project/trainingData.csv")
data_test = pd.read_csv("/content/drive/MyDrive/AI project/validationData.csv")

In [6]:

# Extract features (520 RSSI values)
# Extract output (x, y coordinates)
# Normalize features
# Split into training and test sets
# Normalize outputs

# Extract features (520 RSSI values)
features_training = data_training.iloc[:, :520].replace(100, -110)
features_test = data_test.iloc[:, :520].replace(100, -110)

# Extract output (x, y coordinates)
output_training = data_training[['LATITUDE', 'LONGITUDE', 'FLOOR']]
output_test = data_test[['LATITUDE', 'LONGITUDE', 'FLOOR']]

# Normalize features
scaler_features = MinMaxScaler()
features_training_scaled = scaler_features.fit_transform(features_training)
features_test_scaled = scaler_features.transform(features_test)

# Normalize outputs
scaler_output = MinMaxScaler()
output_training_scaled = scaler_output.fit_transform(output_training)
output_test_scaled = scaler_output.transform(output_test)

# Split into training and test sets
X_train = features_training_scaled
y_train = output_training_scaled
X_test = features_test_scaled
y_test = output_test_scaled

# Network Architecture
## Step 2 : Define SDAE and create Autoencoders

In [7]:
# Create Stacked Denoising Autoencoder (SDAE) model

# Function to create a single denoising autoencoder layer
def SDAE(input_dim, encoding_dim, corruption_level=0.2):
  input_layer = Input(shape=(input_dim,))
  # Add noise
  corrupted_input = Dropout(corruption_level)(input_layer)
  # Encoder
  encoded = Dense(encoding_dim, activation='relu',
                  kernel_regularizer=l2(0.001))(corrupted_input)
  # Decoder
  decoded = Dense(input_dim, activation='sigmoid',
                  kernel_regularizer=l2(0.001))(encoded)
  # Autoencoder model
  autoencoder = Model(inputs=input_layer, outputs=decoded)
  # Encoder model (for stacking)
  encoder_model = Model(inputs=input_layer, outputs=encoded)
  return autoencoder, encoder_model

# Define the dimensions for each autoencoder layer
input_dim = X_train.shape[1] # 520
encoding_dim_1 = 256
encoding_dim_2 = 128
encoding_dim_3 = 64

# Create the first autoencoder
autoencoder_1, encoder_1 = SDAE(input_dim, encoding_dim_1)
autoencoder_1.compile(optimizer='adam', loss='mse')

# Train the first autoencoder
print("Training Autoencoder 1...")
autoencoder_1.fit(X_train, X_train, epochs=50, batch_size=256, shuffle=True,
                  validation_data=(X_test, X_test), verbose=1)

# Get the encoded representation from the first autoencoder for the next layer
X_train_encoded_1 = encoder_1.predict(X_train)
X_test_encoded_1 = encoder_1.predict(X_test)

# Create the second autoencoder
autoencoder_2, encoder_2 = SDAE(encoding_dim_1, encoding_dim_2)
autoencoder_2.compile(optimizer='adam', loss='mse')

# Train the second autoencoder
print("Training Autoencoder 2...")
autoencoder_2.fit(X_train_encoded_1, X_train_encoded_1, epochs=50,
                  batch_size=256, shuffle=True,
                  validation_data=(X_test_encoded_1, X_test_encoded_1),
                  verbose=1)

# Get the encoded representation from the second autoencoder for the next layer
X_train_encoded_2 = encoder_2.predict(X_train_encoded_1)
X_test_encoded_2 = encoder_2.predict(X_test_encoded_1)

# Create the third autoencoder
autoencoder_3, encoder_3 = SDAE(encoding_dim_2, encoding_dim_3)
autoencoder_3.compile(optimizer='adam', loss='mse')

# Train the third autoencoder
print("Training Autoencoder 3...")
autoencoder_3.fit(X_train_encoded_2, X_train_encoded_2, epochs=50,
                  batch_size=256, shuffle=True,
                  validation_data=(X_test_encoded_2, X_test_encoded_2),
                  verbose=1)



Training Autoencoder 1...
Epoch 1/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.6222 - val_loss: 1.6645
Epoch 2/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.1372 - val_loss: 1.6077
Epoch 3/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0967 - val_loss: 1.5887
Epoch 4/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0797 - val_loss: 1.5763
Epoch 5/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0681 - val_loss: 1.5679
Epoch 6/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0597 - val_loss: 1.5615
Epoch 7/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0533 - val_loss: 1.5567
Epoch 8/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0483 - val_loss: 1.5529
Epoch 9/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0443 - val_loss: 1.5499
Epoch 10/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0410 - val_loss: 1.5473
Epoch 11/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0382 - val_loss: 1.5452
Epoch 12/50
78/78 ━━━━━━━━━━━━━━━━━━━

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 520)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_1 (Functional)       │ (None, 256)            │       133,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_3 (Functional)       │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_5 (Functional)       │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 174,723 (682.51 KB)

 Trainable params: 174,723 (682.51 KB)

 Non-trainable params: 0 (0.00 B)

# Step 3: create model

In [ ]:
# Build the full SDAE model by stacking the encoders and adding the final output layer
print("Building Stacked Denoising Autoencoder (SDAE) model...")
sdae_input = Input(shape=(input_dim,))
encoded_1_output = encoder_1(sdae_input)
encoded_2_output = encoder_2(encoded_1_output)
encoded_3_output = encoder_3(encoded_2_output)

# Add dense layers for the final prediction
# (regression for LATITUDE, LONGITUDE, FLOOR)
output_dim = y_train.shape[1] # 3
prediction_output = Dense(output_dim,
                          activation='sigmoid')(encoded_3_output)
                          # Use sigmoid for normalized outputs

sdae_model = Model(inputs=sdae_input, outputs=prediction_output)

# Compile the full SDAE model
sdae_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Print the model summary
sdae_model.summary()

In [8]:
# create model
sdae_input = Input(shape=(input_dim,))
encoded_1 = encoder_1(sdae_input)
encoded_2 = encoder_2(encoded_1)
encoded_3 = encoder_3(encoded_2)

# Decoder reconstruction
decoded_3 = Dense(encoding_dim_2, activation='relu')(encoded_3)
decoded_2 = Dense(encoding_dim_1, activation='relu')(decoded_3)
decoded_output = Dense(input_dim, activation='sigmoid')(decoded_2)  # reconstruction RSSI

sdae_model = Model(inputs=sdae_input, outputs=decoded_output)
sdae_model.compile(optimizer='adam', loss='mse')

sdae_model.summary()


sdae_model.fit(X_train, X_train, epochs=100, batch_size=256,
               validation_data=(X_test, X_test), shuffle=True)


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 520)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_1 (Functional)       │ (None, 256)            │       133,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_3 (Functional)       │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_5 (Functional)       │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 520)            │       133,640 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 349,512 (1.33 MB)

 Trainable params: 349,512 (1.33 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - loss: 0.0730 - val_loss: 1.5273
Epoch 2/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0080 - val_loss: 1.5262
Epoch 3/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0074 - val_loss: 1.5260
Epoch 4/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0071 - val_loss: 1.5259
Epoch 5/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0069 - val_loss: 1.5258
Epoch 6/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0069 - val_loss: 1.5258
Epoch 7/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0069 - val_loss: 1.5258
Epoch 8/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0069 - val_loss: 1.5258
Epoch 9/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0068 - val_loss: 1.5258
Epoch 10/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0068 - val_loss: 1.5259
Epoch 11/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0068 - val_loss: 1.5259
Epoch 12/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step

In [9]:
X_test_reconstructed = sdae_model.predict(X_test)


35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


# Step 4: Calculation error

In [10]:
reconstruction_error = mean_squared_error(X_test.flatten(), X_test_reconstructed.flatten())
print("Reconstruction MSE:", reconstruction_error)


Reconstruction MSE: 1.5259210942287946


# Localization Estimator
## compare reconstructed RSS and fingerprins

In [11]:
# Comparison operation
def compute_similarity(x_input, x_reconstructed, sigma=1.0):
    distance = np.linalg.norm(x_input - x_reconstructed)
    similarity = np.exp(-distance / sigma)
    return similarity
#location prediction
def probabilistic_localization(test_sample, sdae_model, fingerprint_db, locations, sigma=1.0):
    reconstructed = sdae_model.predict(test_sample.reshape(1, -1))[0] # prediction
    sims = [compute_similarity(fp, reconstructed, sigma) for fp in fingerprint_db]
    sims = np.array(sims)
    sims /= sims.sum()  # normalize to 1
    est_location = np.sum(locations * sims[:, np.newaxis], axis=0)  # weighted average
    return est_location


# Step 5: Evaluate model

In [12]:
fingerprint_db = X_train
fingerprint_locations = y_train

estimated_locations = []
for i in range(X_test.shape[0]):
    est_loc = probabilistic_localization(X_test[i], sdae_model, fingerprint_db, fingerprint_locations)
    estimated_locations.append(est_loc)

estimated_locations = np.array(estimated_locations)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━

In [13]:
errors = np.linalg.norm(y_test - estimated_locations, axis=1)  # Euclidean distance
mean_error = np.mean(errors)
print("Mean Localization Error (meters):", mean_error)


Mean Localization Error (meters): 0.5179394086922522
